In [ ]:
from mt import *

In [ ]:
test()

Applied a rolling ball of radius 15 to the image before loading.

In [ ]:
# The directory containing the images
data_dir = '/Volumes/martin/200116_tandem_retina_explants/27-T3/ori/14/bgsub'

# glob string for images
im_glob_dapi = os.path.join(data_dir, '200116_tandem_bAA27_T3_1_w1*')
im_list_dapi = sorted(glob.glob(im_glob_dapi))
im_glob_cfp = os.path.join(data_dir, '200116_tandem_bAA27_T3_1_w2*')
im_list_cfp = sorted(glob.glob(im_glob_cfp))
im_glob_r = os.path.join(data_dir, '200116_tandem_bAA27_T3_1_w3*')
im_list_r = sorted(glob.glob(im_glob_r))
im_glob_fr = os.path.join(data_dir, '200116_tandem_bAA27_T3_1_w4*')
im_list_fr = sorted(glob.glob(im_glob_fr))

im_list_fr

In [ ]:
i = 1
dapi_raw = skimage.io.imread(im_list_dapi[i])
cfp_raw = skimage.io.imread(im_list_cfp[i])
r_raw = skimage.io.imread(im_list_r[i])
fr_raw = skimage.io.imread(im_list_fr[i])

In [ ]:
cfp, r, fr = cfp_raw, r_raw, fr_raw

In [ ]:
cfp_adj = imadjust(cfp, lower_bound=0.1, upper_bound=99.9)
r_adj = imadjust(r, lower_bound=0.1, upper_bound=99.9)
fr_adj = imadjust(fr, lower_bound=0.1, upper_bound=99.9)

In [ ]:
bokeh.io.show(show_three_ims(cfp_adj[14], r_adj[14], fr_adj[14]))

In [ ]:
fr_adj = register_images(r_adj, fr_adj)

In [ ]:
cfp_adj_g, r_adj_g, fr_adj_g = filt_gauss(cfp_adj, r_adj, fr_adj, 3)

In [ ]:
print(threshold_multiotsu(cfp_adj_g, classes=4))
cfp_mo0 = cfp_adj_g > threshold_multiotsu(cfp_adj_g, classes=4)[0]
cfp_mo1 = cfp_adj_g > threshold_multiotsu(cfp_adj_g, classes=4)[2]

print(threshold_multiotsu(fr_adj_g, classes=3))
fr_mo0 = fr_adj_g > threshold_multiotsu(fr_adj_g)[0]
fr_mo1 = fr_adj_g > threshold_multiotsu(fr_adj_g)[1]

print(threshold_multiotsu(r_adj_g, classes=4))
r_mo0 = r_adj_g > threshold_multiotsu(r_adj_g, classes=4)[0]
r_mo1 = r_adj_g > threshold_multiotsu(r_adj_g, classes=4)[2]

In [ ]:
bokeh.io.show(show_three_ims(cfp_adj[14], cfp_mo0[14], cfp_mo1[14]))

The lowest threshold for CFP is the best because it captures the bright and dim nuclei as well.

In [ ]:
cfp_small = skimage.morphology.remove_small_objects(cfp_mo0, 4500)

In [ ]:
bokeh.io.show(show_three_ims(cfp_adj[13], cfp_mo0[13], cfp_small[13]))

In [ ]:
bokeh.io.show(show_six_ims(fr_adj[14], fr_mo0[14], fr_mo1[14], r_adj[14], r_mo0[14], r_mo1[14]))

The highest threshold for dots works best because it gets the brightest dots (the active sites) and NOT the single molecules.

In [ ]:
fr_small = skimage.morphology.remove_small_objects(fr_mo1, 50)
r_small = skimage.morphology.remove_small_objects(r_mo1, 50)

200 and 100 removes active sites. 50 seems alright.

In [ ]:
i = 14
bokeh.io.show(show_six_ims(fr_adj[i], fr_mo1[i], fr_small[i], r_adj[i], r_mo1[i], r_small[i]))

In [ ]:
selem = skimage.morphology.ball(3)

# Erode binary image
cfp_bw_opened = skimage.morphology.binary_opening(cfp_small, selem)

In [ ]:
label_image_cfp, cfp_dist, cfp_mrkr = ws_nuc(cfp_bw_opened)

In [ ]:
i=10
bokeh.io.show(show_six_ims(cfp_adj[i], cfp_small[i], cfp_bw_opened[i], 
                           cfp_dist[i], cfp_mrkr[i], label_image_cfp[i],
                           cmap=[None, None, None, 
                                 colorcet.gray, colorcet.gray, colorcet.b_glasbey_hv]))

This is to test different min_sizes for remove_small_object.

In [ ]:
label_image_cfp_small, df_cfp, cfp_area = remove_small_df(label_image_cfp, 3000, cfp)
label_image_cfp_small2, df_cfp2, cfp_area2 = remove_small_df(label_image_cfp, 4500, cfp)

Tested 5000, seems too harsh, starts removing actual nuclei. 4500 seems good.

In [ ]:
i=10
bokeh.io.show(show_six_ims(cfp_adj[i], cfp_small[i], cfp_bw_opened[i], 
                           label_image_cfp[i], label_image_cfp_small[i], label_image_cfp_small2[i],
                           cmap=[None, None, None, 
                                 colorcet.b_glasbey_hv, colorcet.b_glasbey_hv, colorcet.b_glasbey_hv]))

In [ ]:
selem = skimage.morphology.ball(5)

# Dilate image
label_image_cfp_small_dil = skimage.morphology.dilation(label_image_cfp_small2, selem)

In [ ]:
i=14
bokeh.io.show(show_six_ims(cfp_adj[i], cfp_small[i], cfp_bw_opened[i], 
                           label_image_cfp[i], label_image_cfp_small2[i], label_image_cfp_small_dil[i],
                           cmap=[None, None, None, 
                                 colorcet.b_glasbey_hv, colorcet.b_glasbey_hv, colorcet.b_glasbey_hv]))

In [ ]:
label_image_fr, dist_fr, mrkr_fr = ws_dot(fr_small)
label_image_r, dist_r, mrkr_r = ws_dot(r_small)

In [ ]:
i=10
bokeh.io.show(show_six_ims(fr_adj[i], fr_small[i], label_image_fr[i], r_adj[i], r_small[i], label_image_r[i], 
                            titles=['original', 'seg', 'ws', 'original', 'seg', 'ws',], 
                             cmap=[None, None, colorcet.b_glasbey_hv, None, None, colorcet.b_glasbey_hv]))

In [ ]:
label_image_fr_small, df_fr, fr_area = remove_small_df(label_image_fr, 50, fr)
label_image_r_small, df_r, r_area = remove_small_df(label_image_r, 50, r)

In [ ]:
i=10
bokeh.io.show(show_six_ims(fr_adj[i], fr_small[i], label_image_fr_small[i], 
                           r_adj[i], r_small[i], label_image_r_small[i], 
                            titles=['original', 'seg', 'ws', 'original', 'seg', 'ws',], 
                             cmap=[None, None, colorcet.b_glasbey_hv, None, None, colorcet.b_glasbey_hv]))

# Colocalization

Now that I have finished segmenting my nuclei and dots, I will colocalize them.

In [ ]:
df_cfp2

In [ ]:
hv.Histogram(
    data=np.histogram(df_cfp2['area'], bins=50),
    kdims=['area']
)

I will make a mask of my red dots and my far-red dots. Then I will make an OR mask and find the CFP+ nuclei that have any dots.

In [ ]:
r_c0 = df_r['centroid-0'].values
r_c1 = df_r['centroid-1'].values
r_c2 = df_r['centroid-2'].values

r_centroids = np.zeros_like(r_mo1)

for i, (a, b, c) in enumerate(zip(r_c0, r_c1, r_c2)):
    r_centroids[a, b, c] = 1

In [ ]:
fr_c0 = df_fr['centroid-0'].values
fr_c1 = df_fr['centroid-1'].values
fr_c2 = df_fr['centroid-2'].values

fr_centroids = np.zeros_like(fr_mo1)

for i, (a, b, c) in enumerate(zip(fr_c0, fr_c1, fr_c2)):
    fr_centroids[a, b, c] = 1

In [ ]:
r_or_fr = np.logical_or(r_centroids, fr_centroids)

In [ ]:
pos_nuc = label_image_cfp_small_dil * r_or_fr

labels_pos_nuc = skimage.measure.regionprops_table(pos_nuc, properties=('label','centroid'))

df_pos_nuc = pd.DataFrame(labels_pos_nuc)

In [ ]:
df_pos_nuc['co_dots'] = [0 for i in range(len(df_pos_nuc))]
df_pos_nuc['r_dots'] = [0 for i in range(len(df_pos_nuc))]
df_pos_nuc['fr_dots'] = [0 for i in range(len(df_pos_nuc))]

In [ ]:
df_pos_nuc

In [ ]:
print('Fraction of CFP nuclei with any barcodes detected:', len(df_pos_nuc)/len(df_cfp2))

I will create a mask with only the CFP+ nuclei that have any barcodes.

In [ ]:
label_image_pos_nuc = np.zeros_like(label_image_cfp_small_dil)

for i, label in enumerate(df_pos_nuc['label'].values):
    bool_array = label_image_cfp_small_dil == label
    single_label = bool_array * label
    label_image_pos_nuc += single_label

In [ ]:
i=18
bokeh.io.show(show_four_ims(label_image_fr[i], label_image_r[i], 
                           label_image_cfp_small_dil[i], label_image_pos_nuc[i],
                            titles=['far-red dot', 'red dot', 'all cfp', 'cfp w/ dot'], 
                             cmap=[colorcet.b_glasbey_hv, colorcet.b_glasbey_hv, 
                                   colorcet.b_glasbey_hv, colorcet.b_glasbey_hv]))

Next, I will classify my dots. I will start with far red dots, using their centroids that are taken directly from the watershed labels. 

For each dot, I will calculate the distance to every red dot. I will count the dot as "co-localized" with a red dot if their centroids are less than 4 pixels apart. If it is co-localized, I will store this in a list with all co-localized dots; if it is not, I will store it in a list with far red **only** dots.

In [ ]:
co_dot, fr_dot = classify_dots(fr_c0, fr_c1, fr_c2, r_c0, r_c1, r_c2, 4)
co_dot2, r_dot = classify_dots(r_c0, r_c1, r_c2, fr_c0, fr_c1, fr_c2, 4)

There are no dots with multiple dots nearby, which is good.

In [ ]:
if len(co_dot) != len(co_dot2):
    print('error')
else:
    print(len(co_dot))

Using the far red dots or the red dots gives me the same amount of co-localized dots as expected. 

In [ ]:
len(fr_dot)

In [ ]:
len(r_dot)

Now that I have my dots classified as red only, far-red only, or co-localized, I will apply a CFP mask to them to only look at the ones within CFP+ nuclei. First I will put the dots centroids into a DataFrame, starting with far-red dots. 

In [ ]:
df_fr_dot = make_df(fr_dot)
df_r_dot = make_df(r_dot)
df_co_dot = make_df(co_dot)
df_co_dot2 = make_df(co_dot2)

Next I will use the centroid positions and multiply this by the CFP mask to obtain all dots within CFP+ nuclei.

In [ ]:
df_fr_nuc = dots_to_nuc(df_fr_dot, fr_mo1, label_image_cfp_small_dil)
df_r_nuc = dots_to_nuc(df_r_dot, fr_mo1, label_image_cfp_small_dil)
df_co_nuc = dots_to_nuc(df_co_dot, fr_mo1, label_image_cfp_small_dil)
df_co_nuc2 = dots_to_nuc(df_co_dot2, fr_mo1, label_image_cfp_small_dil)

There is an equal amount of colocalized dots either way I look - which is good.

Next I will assign each dot to a CFP+ nuclei label. I will do this for the colocalized, red only, and far-red only dots.

In [ ]:
df_pos_nuc = df_pos_nuc.set_index('label')

In [ ]:
co_nuc0 = df_co_nuc['centroid-0'].values
co_nuc1 = df_co_nuc['centroid-1'].values
co_nuc2 = df_co_nuc['centroid-2'].values

for i, (a, b, c) in enumerate(zip(co_nuc0, co_nuc1, co_nuc2)):
    label = label_image_cfp_small_dil[a, b, c]
    df_pos_nuc.at[label, 'co_dots'] += 1

In [ ]:
r_nuc0 = df_r_nuc['centroid-0'].values
r_nuc1 = df_r_nuc['centroid-1'].values
r_nuc2 = df_r_nuc['centroid-2'].values

for i, (a, b, c) in enumerate(zip(r_nuc0, r_nuc1, r_nuc2)):
    label = label_image_cfp_small_dil[a, b, c]
    df_pos_nuc.at[label, 'r_dots'] += 1

In [ ]:
fr_nuc0 = df_fr_nuc['centroid-0'].values
fr_nuc1 = df_fr_nuc['centroid-1'].values
fr_nuc2 = df_fr_nuc['centroid-2'].values

for i, (a, b, c) in enumerate(zip(fr_nuc0, fr_nuc1, fr_nuc2)):
    label = label_image_cfp_small_dil[a, b, c]
    df_pos_nuc.at[label, 'fr_dots'] += 1

In [ ]:
df_pos_nuc['sum'] = df_pos_nuc['co_dots'].values + df_pos_nuc['r_dots'].values + df_pos_nuc['fr_dots'].values

In [ ]:
co_dots_sum = df_pos_nuc['co_dots'].values.sum()
r_dots_sum = df_pos_nuc['r_dots'].values.sum()
fr_dots_sum = df_pos_nuc['fr_dots'].values.sum()

print('co_dots:', co_dots_sum)
print('r_dots:', r_dots_sum)
print('fr_dots:', fr_dots_sum)

In [ ]:
df_pos_nuc

In [ ]:
df_counts = df_pos_nuc['sum'].value_counts(
).reset_index(
).rename(
    columns={"index": "sum", "sum": "count"}
)

# Take a look
df_counts

In [ ]:
# need to add zeros for the CFP+ nuclei with no barcodes
zeros_num = len(df_cfp2) - len(df_pos_nuc['sum'].values)
zeros = [0 for i in range(zeros_num)]

In [ ]:
data = {'sum': [0], 'count': [zeros_num]}
df_zeros = pd.DataFrame(data)
df_zeros

In [ ]:
df_counts_zero = df_counts.append(df_zeros)
df_counts_zero

In [ ]:
spikes = hv.Spikes(
    data=df_counts_zero,
    kdims=['sum'],
    vdims=['count']
).opts(
    line_width=20
)

spikes

In [ ]:
i=20
test = label_image_cfp_small_dil == 62
bokeh.io.show(show_four_ims(fr_adj[i], r_adj[i], cfp_adj[i], test[i], 
                             cmap=[None, None, None, colorcet.b_glasbey_hv]))

In [ ]:
dot_sum = df_pos_nuc['sum'].values.tolist()
total = dot_sum + zeros

In [ ]:
silenced = (len(co_dot) - len(df_co_nuc)) / len(co_dot)
print('Fraction of nuclei with silenced CFP:', silenced)

In [ ]:
any_barcode = df_counts['count'].values.sum()/len(total)
one_barcode = df_counts['count'].values[0]/len(total)
one_plus_barcode = df_counts['count'].values[1:].sum()/len(total)
print('Fraction of CFP nuclei with any barcodes detected:', any_barcode)
print('Fraction of CFP nuclei with 1 barcode detected:', one_barcode)
print('Fraction of CFP nuclei with more than 1 barcode detected:', one_plus_barcode)

In [ ]:
data = [('s9', silenced)]
bars = hv.Bars(data, hv.Dimension('Position'), 'Fraction of silenced CFP nuclei').opts(ylim=(0, 1))
bars